In [1]:
animal = 'AE2'
unit = '001'
trials = ['047', '048', '049']

file_path = 'D:\\2P_data\\F\\%s\\u%s_%s\\1_2_3\\F_%s_u%s_%s_plane1_proc.mat' % (animal, unit, trials[0], animal, unit, trials[0])
analyzer_paths = []
for trial_ in trials:
    analyzer_paths.append('H:\\2P_nhp_test\\AnalyzerFiles\\%s\\%s_u%s_%s.analyzer' % (animal, animal, unit, trial_))
plot = 'y'
plot_gaussian_fit = 'n'

In [33]:
import pandas as pd
import scipy.io as sio
import functions

F = sio.loadmat(file_path, squeeze_me=True, struct_as_record=False)['dat']
params = []
trial_num_list = []
for i, analyzer_path_ in enumerate(analyzer_paths):
    params.append(functions.analyzer_params(analyzer_path_))
    trial_num_, stim_time = functions.analyzer_pg_conds(analyzer_path_)
    trial_num_['colormod'] = params[i]['colormod']
    trial_num_list.append(trial_num_)
trial_num = pd.concat(trial_num_list, ignore_index=True)
trial_num['direction'] = trial_num.ori
trial_num.ori[(trial_num.ori >= 180)] = trial_num.ori[(trial_num.ori >= 180)] - 180

C:\Users\anupam\Miniconda3\envs\color_vision\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [67]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

mean_image = F.mimg[:, :, 1]
color_mask = np.zeros(mean_image.shape)
pref_cells = pd.DataFrame(columns=['expt', 'x', 'y', 'color_pref'])

for i, cell in enumerate(F.stat):
    if cell.iscell == 1:
        scans = int(np.concatenate(F.Fcell, axis=1).shape[1] / trial_num.shape[0])
        t = np.linspace(stim_time[0] * -1, stim_time.sum() - stim_time[1], scans)
        cell_F = pd.DataFrame(np.reshape(np.concatenate(F.Fcell, axis=1)[i, :], (int(np.concatenate(F.Fcell, axis=1).shape[1] / scans), scans)))
        cell_F.insert(0, 'Orientation', trial_num.ori)
        cell_F.insert(1, 'Direction', trial_num.direction)
        cell_F.insert(2, 'Spatial Frequency', trial_num.s_freq)
        cell_F.insert(3, 'Color', trial_num.colormod)
        cell_F.insert(4, 'Baseline',
                      np.reshape(np.concatenate(F.Fcell, axis=1)[i, :], (int(np.concatenate(F.Fcell, axis=1).shape[1] / scans), scans))
                      [:, 0:int(np.floor(scans * stim_time[0] / stim_time.sum())) - 1].mean(axis=1))
        
        start_sample = int(np.ceil(scans * (stim_time[0] + 0.5) / stim_time.sum()))
        end_sample = int(scans - np.ceil(scans * (stim_time[1]) / stim_time.sum()))
        
        cell_F.iloc[:, -scans:] = cell_F.iloc[:, -scans:].subtract(cell_F['Baseline'], axis=0).divide(cell_F['Baseline'],
                                                                                                  axis=0)
        
        cell_F_stim = pd.DataFrame(cell_F.iloc[:, -(scans-start_sample):-(scans-end_sample)].mean(axis=1), columns=['Mean'])
        cell_F_stim.insert(0, 'Orientation', trial_num.ori)
        cell_F_stim.insert(1, 'Direction', trial_num.direction)
        cell_F_stim.insert(2, 'Spatial Frequency', trial_num.s_freq)
        cell_F_stim.insert(3, 'Color', trial_num.colormod)
        
        max_response = cell_F_stim.groupby('Color').mean()['Mean'].max()
        
        colors = np.unique(trial_num.colormod.values)
        list_F_stim = [cell_F_stim[cell_F_stim['Color'] == color]['Mean'] for color in colors]
        F_value, p = sp.stats.f_oneway(*list_F_stim)
        
        if p < 0.001:
            max_response = cell_F_stim.groupby('Color').mean()['Mean'].max()
            if max_response > 0.03:
                max_color = cell_F_stim.groupby('Color').mean()['Mean'].idxmax()
                # if max_color < 16:
                #     ortho_color = max_color + 6
                # else:
                #     ortho_color = max_color - 6
                # ortho_response = cell_F_stim.groupby('Color').mean()['Mean'][ortho_color]
                # color_si = (max_response - ortho_response) / max_response
                # if color_si > 0.7:
                color_mask[(cell.ypix - 1), (cell.xpix - 1)] = max_color
                if plot == 'y':
                    plt.ioff()
                    plt.figure()
                    plt.plot(cell_F_stim.groupby('Color').mean()['Mean'])
                    plt.xlabel('Colormod')
                    plt.ylabel(r'Response ($\Delta F/F$)')
                    plt.savefig('H:/nhp_cells/cell%d.pdf' % i, format='pdf')
                    plt.close()
                x_med, y_med = cell.med
                pref_cells.loc[i] = [analyzer_paths[0][-17:-9], x_med, y_med, max_color]

pref_cells.to_csv('H:/nhp_cells/%s_color_pref_df.csv' % analyzer_paths[0][-17:-9])

In [66]:
int(np.concatenate(F.Fcell, axis=1).shape[1] / trial_num.shape[0])

24

In [34]:
cell_F.tail()

,0,1,2,3,4,5,6,7
595,68.872475,69.306404,70.643883,78.908730,68.459229,62.847580,60.069637,69.381401
596,60.520477,71.359825,68.275803,64.667450,66.759521,67.043068,66.806801,72.266823
597,73.033722,67.930374,65.261543,61.529476,71.040443,67.996178,67.543892,63.862896
598,68.023659,60.415890,85.586975,69.335045,69.371536,71.919846,66.430267,73.611031
599,74.730392,70.599228,69.897293,62.541679,63.807125,72.532166,63.265617,66.388702


In [36]:
trial_num.ori.tail()

595    45.0
596    90.0
597    90.0
598    45.0
599    45.0
Name: ori, dtype: float64